<img src="https://uploads-ssl.webflow.com/614b1fe22fa8b90ef41aeffe/6265cb48f9496b1cefc9ab75_logotipo-mbit-39.png" width="200px" align="right" CLASS="TextWrap" style="background-color:#2a3f3f;">

</br>
</br>

## Cómo crear aplicaciones visuales para interactuar con tus datos

---

### Streamlit

---

#### Javier Cózar (javier.cozar@mbitschool.com)

---

## Datos

En esta libreta vamos a trabajar con unos datos obtenidos de [Kaggle](https://www.kaggle.com/datasets/carrie1/ecommerce-data?select=data.csv) relacionados con datos del un e-commerce, y datos de latitud y longitud obtenidos a partir de [Google](https://developers.google.com/public-data/docs/canonical/countries_csv). Estos datos han sido combinados y transformados a formato parquet.

[Streamlit](https://streamlit.io/) es una librería de Python que nos permite desarrollar aplicaciones visuales con un mínimo de código. No requiere experiencia en front-end, ya que encapsula la estructura visual de la aplicación en funciones interpretables de Python.

## st.write

Esta funcionalidad de streamlit permite visualizar cualquier cosa en la aplicación, desde un texto hasta una tabla de pandas o un plot! Vamos a comprobarlo.

**Nota**: para lanzar nuestra aplicación basta con escribir `streamlit run <script.py>`, donde script.py es un fichero escrito en python (copiaremos el código de las celdas al script de python).

In [1]:
import streamlit as st
import pandas as pd

df = pd.read_parquet("ecommerce.parquet")

In [7]:
df_plot = (
    df
    .groupby("Country")
    .agg({
        "Quantity": "sum"
    })
    .reset_index()
)

In [ ]:
df_plot

,Country,Quantity
0,Australia,83653
1,Austria,4827
2,Bahrain,260
3,Belgium,23152
4,Brazil,356
5,Canada,2763
6,Cyprus,6317
7,Czech Republic,592
8,Denmark,8188
9,Finland,10666


In [8]:
import altair as alt

In [9]:
alt.Chart(df_plot).mark_bar().encode(x="Quantity", y="Country")

alt.Chart(...)

In [99]:
import streamlit as st
import pandas as pd

df = pd.read_parquet("ecommerce.parquet")

st.write("This is a pandas dataframe:")
st.write(df.head())

## Develompent mode

Cada vez que se altera el script de python que streamlit está ejecutando, éste lo detecta y permite recargar la página automáticamente. Esta opción está en el menú superior derecho, en _Settings_, y en _Run on save_.

In [11]:
import streamlit as st
import pandas as pd

df = pd.read_parquet("ecommerce.parquet")

## Funciones específicas

La función `st.write` es mágica, en el sentido de que renderiza lo que sea que le pasemos (texto, dataframes, e incluso plots como podemos ver a continuación!

In [16]:
import altair as alt

df_plot = (
    df
    .assign(
        total=lambda df: df.Quantity * df.UnitPrice
    )
    .groupby("Country")
    .agg({
        "total": "sum"
    })
    .reset_index()
)

alt.Chart(df_plot).mark_bar().encode(x="Country", y="total")

alt.Chart(...)

Realmente lo que hace esta función es analizar el tipo de datos del argumento y llamar a una función específica de streamlit para renderizarlo. Por ejemplo, para renderizar un dataframe de pandas llama internamente a `st.dataframe`.

La ventaja de llamar a las funciones espexíficas directamente es que podemos personalizar mucho más cómo se renderizan los elementos.

Por ejemplo, vamos a mostrar a través del siguiente código una tabla que muestra estadísticos por país, resaltanto los valores máximos en gris:

In [12]:
df_table = (
    df
    .assign(
        total=lambda df: df.Quantity * df.UnitPrice
    )
    .groupby("Country")
    .agg({
        "total": "sum",
        "Quantity": "sum",
        "UnitPrice": "mean"
    })
)

df_table.style.highlight_max(color="lightgray")

,total,Quantity,UnitPrice
Country,,,
Australia,137077.270000,83653,3.220612
Austria,10154.320000,4827,4.243192
Bahrain,548.400000,260,4.644118
Belgium,40910.960000,23152,3.644335
Brazil,1143.600000,356,4.456250
Canada,3666.380000,2763,6.030331
Cyprus,12946.290000,6317,6.302363
Czech Republic,707.720000,592,2.938333
Denmark,18768.140000,8188,3.256941


Si en lugar de usar `st.write` usamos `st.dataframe` podemos limitar la anchura y altura máxima:

In [97]:
st.dataframe(df_table.style.highlight_max(color="lightgray"), height=200)

DeltaGenerator(_root_container=0, _provided_cursor=None, _parent=None, _block_type=None, _form_data=None)

Si no queremos que la tabla sea interactiva, por ejemplo podemos ordenar de acuerdo a una columna y generar una tabla estática con `st.table`:

In [98]:
st.table(df_table.sort_values("total", ascending=False).head())

DeltaGenerator(_root_container=0, _provided_cursor=None, _parent=None, _block_type=None, _form_data=None)

In [14]:
all_countries = df.Country.unique()
st.selectbox("Select a country", options=all_countries)

2022-05-31 20:02:32.270 
  command:

    streamlit run /Users/jcozar/dev/Docencia/MBIT/2022-05-Webinar-Streamlit/venv/lib/python3.9/site-packages/ipykernel_launcher.py [ARGUMENTS]


'United Kingdom'

En la documentación podemos encontrar multitud de [elementos](https://docs.streamlit.io/library/api-reference) a renderizar en una aplicación de Streamlit! Vamos a explorar algunos de ellos:

- Title
- Markdown
- Code
- Metrics
- Matplotlib

¡También podemos interactuar con nuestra aplicación! Para ello disponemos de elementos como listas desplegables, sliders, text inputs, o incluso selectores de fechas.

### ejemplo 1 - elementos visuales en Streamlit

In [ ]:
import streamlit as st
import pandas as pd
import seaborn as sns


st.title("Cómo crear aplicaciones visuales para interactuar con tus datos")

st.markdown("La columna `total` refleja `Quantity` $\cdot$ `UnitPrice`. Para construirla con Python se ha usado el siguiente código:")

st.code("""
(
    df
    .assign(
        total=lambda df: df.Quantity * df.UnitPrice
    )
)
""")

df = pd.read_parquet("ecommerce.parquet")               
               
df_table = (
    df
    .assign(
        total=lambda df: df.Quantity * df.UnitPrice
    )
    .groupby("Country")
    .agg({
        "total": "sum",
        "Quantity": "sum",
        "UnitPrice": "mean"
    })
)

fg = sns.catplot(data=df_table.reset_index(), kind="bar", y="Country", x="total", aspect=3)
st.pyplot(fg.fig)

all_countries = sorted(df.Country.unique())
selected_country = st.selectbox("Selected countries", options=all_countries)

st.table(df_table.loc[selected_country])

q = df_table.loc[selected_country].Quantity

st.metric(f"{selected_country} quantity", q, delta=q-df_table.Quantity.mean())


### ejemplo 2 - interactuando con inputs

In [ ]:
import streamlit as st
import pandas as pd
import altair as alt
from vega_datasets import data


st.title("Cómo crear aplicaciones visuales para interactuar con tus datos")

df = (
    pd.read_parquet("ecommerce.parquet")               
    .assign(
        InvoiceDate=lambda df: pd.to_datetime(df.InvoiceDate)
    )
)
               
min_date = df.InvoiceDate.min()
max_date = df.InvoiceDate.max()
min_date_input = st.date_input("Minimum date for invoices", min_date, min_value=min_date, max_value=max_date)
max_date_input = st.date_input("Maximum date for invoices", max_date, min_value=min_date, max_value=max_date)

min_date_input = min_date_input.strftime("%Y-%m-%d")
max_date_input = max_date_input.strftime("%Y-%m-%d")

df_show = (
    df
    .loc[lambda df: (df.InvoiceDate >= min_date_input) & (df.InvoiceDate <= max_date_input)]
)

q = st.slider("Minimum quantity", min_value=0, max_value=int(df_show.Quantity.max()))

df_show = (
    df_show.loc[lambda df: (df.Quantity >= q)]
)

st.dataframe(df_show)


# Map plot
df_plot = (
    df_show
    .assign(
        total=lambda df: df.Quantity * df.UnitPrice
    )
    .groupby("Country")
    .agg({
        "total": "sum",
        "latitude": "first",
        "longitude": "first"
    })
    .reset_index()
)


source = alt.topo_feature(data.world_110m.url, "countries")
base_map = (
    alt.Chart(source)
    .mark_geoshape(fill="white", stroke="gray")
    .properties(width=900, height=500)
    .project("naturalEarth1")
)

points = (
    alt.Chart(df_plot)
    .mark_point()
    .encode(
        latitude="latitude",
        longitude="longitude",
        fill=alt.value("red"),
        size=alt.Size("total:Q", scale=alt.Scale(type='log')),  # linear, log
        stroke=alt.value(None),
     )
)

final_map = (
    (base_map + points)
    .configure_view(strokeWidth=0)
    .configure_mark(opacity=0.5,)
)

st.altair_chart(final_map)

## Layouts

Una de las principales ventajas de streamlit es que no nos preocupamos del layout, simplemente indico lo que quiero visualizar ¡y ya se encarga el módulo!

Pero cuando la aplicación tiene cierta entidad se vuelve indispensable poder **organizar** los elementos. La estrategia de streamlit es muy acertada: **dividir la aplicación en secciones y permitir introducir los elementos en cada una de ellas** mediante funciones concretas. Podemos encontrar la documentación [aquí](https://docs.streamlit.io/library/api-reference/layout).

A continuación vamos a usar dos tipos de layout:

- sidebar: lo usaremos para situar los filtros de nuestra aplicación
- columns: el contenido de nuestra aplicación se renderizará en la zona principal, si necesitamos estructurarlo en varias columnas podemos hacerlo con `st.columns`.

### Sidebar

![image](https://docs.streamlit.io/images/api/sidebar.jpg)

### Columns

![image](https://docs.streamlit.io/images/api/columns.jpg)

### ejemplo 3 - ejemplo con layouts

In [ ]:
import streamlit as st
import pandas as pd
import altair as alt
from vega_datasets import data


st.title("Cómo crear aplicaciones visuales para interactuar con tus datos")

df = (
    pd.read_parquet("ecommerce.parquet")               
    .assign(
        total=lambda df: df.Quantity * df.UnitPrice,
        InvoiceDate=lambda df: pd.to_datetime(df.InvoiceDate)
    )
)

st.sidebar.title("Filtros para el dataframe")

min_date = df.InvoiceDate.min()
max_date = df.InvoiceDate.max()
min_date_input = st.sidebar.date_input("Minimum date for invoices", min_date, min_value=min_date, max_value=max_date)
max_date_input = st.sidebar.date_input("Maximum date for invoices", max_date, min_value=min_date, max_value=max_date)

min_date_input = min_date_input.strftime("%Y-%m-%d")
max_date_input = max_date_input.strftime("%Y-%m-%d")

df_show = (
    df
    .loc[lambda df: (df.InvoiceDate >= min_date_input) & (df.InvoiceDate <= max_date_input)]
)

up = st.sidebar.slider("Minimum unit price", min_value=0, max_value=int(df_show.UnitPrice.max()))

df_show = (
    df_show.loc[lambda df: (df.UnitPrice >= up)]
)


q = st.sidebar.slider("Minimum quantity", min_value=0, max_value=int(df_show.Quantity.max()))

df_show = (
    df_show.loc[lambda df: (df.Quantity >= q)]
)

st.dataframe(df_show, height=200)


# Map plot
df_plot = (
    df_show
    .assign(
        total=lambda df: df.Quantity * df.UnitPrice
    )
    .groupby("Country")
    .agg({
        "total": "sum",
        "Quantity": "sum",
        "latitude": "first",
        "longitude": "first"
    })
    .reset_index()
)

map_scale = st.sidebar.selectbox("Map quantity scale", ["log", "linear"])

source = alt.topo_feature(data.world_110m.url, "countries")
base_map = (
    alt.Chart(source)
    .mark_geoshape(fill="white", stroke="gray")
    .properties(width=900, height=500)
    .project("naturalEarth1")
)

points = (
    alt.Chart(df_plot)
    .mark_point()
    .encode(
        latitude="latitude",
        longitude="longitude",
        fill=alt.value("red"),
        size=alt.Size("total:Q", scale=alt.Scale(type=map_scale)),
        stroke=alt.value(None),
     )
)

final_map = (
    (base_map + points)
    .configure_view(strokeWidth=0)
    .configure_mark(opacity=0.5,)
)

st.altair_chart(final_map)



col1, col2 = st.columns(2)

with col1:
    st.altair_chart(alt.Chart(df_plot).mark_bar().encode(y="Country", x="total"))

with col2:
    st.altair_chart(alt.Chart(df_plot).mark_bar().encode(y="Country", x="Quantity"))